# XGBoost sur données brutes
## Création model et entrainement

In [1]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
data_path = "data/train_1_sample.csv"
data = pd.read_csv(data_path)


data = data[(data['pickup_latitude'] != 0) | (data['pickup_longitude'] != 0) | (data['dropoff_latitude'] != 0) | (data['dropoff_longitude'] != 0)]
data = data[(data['pickup_latitude'] <=90)
            & (data['pickup_latitude'] >=-90)
            & (data['pickup_longitude'] <=90)
            & (data['pickup_longitude'] >=-90)
            & (data['dropoff_latitude'] <=90)
            & (data['dropoff_latitude'] >=-90)
            & (data['dropoff_longitude'] <=90)
            & (data['dropoff_longitude'] >=-90)]




In [3]:
date = data.iloc[:, 2]
date = pd.DatetimeIndex(date)

In [4]:

data["dayofweek"] = date.dayofweek
data["day"] = date.day
data["month"] = date.month
data["year"] = date.year
data["hour"] = date.hour


In [64]:
data_grouped = data.groupby(["year", "month", "dayofweek", "hour"]).mean().loc[:, ['fare_amount']]
#data_grouped = data_grouped.reset_index(level=data_grouped.index.names)
data_grouped

fare_amount
year month dayofweek hour             
2009 1     0         0       10.115714
                     1       10.120000
                     2       10.827273
                     3       10.363636
                     4       12.712500
...                                ...
2015 6     6         19      11.501311
                     20      13.282807
                     21      12.771045
                     22      13.138824
                     23      15.026296

[13104 rows x 1 columns]

In [65]:
Y = data.iloc[:, 1]
X = data.iloc[:, 3:]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
Y_train

361320     9.5
736207     5.3
223908    10.1
905775    13.0
527373     9.5
          ... 
189970     6.0
125805     4.1
531478     8.0
329588     4.0
651405     7.3
Name: fare_amount, Length: 785621, dtype: float64

In [66]:
Y_pred = X_train.join(data_grouped, on=['year', 'month', 'dayofweek', 'hour'], how="left")
Y_pred

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dayofweek,day,month,year,hour,fare_amount
361320,-73.990012,40.725420,-74.002815,40.706442,1,5,31,8,2013,14,10.569620
736207,-73.988367,40.740970,-73.994953,40.750312,1,1,24,11,2009,16,11.361594
223908,-73.972995,40.764583,-74.001038,40.729315,5,2,24,6,2009,20,9.342135
905775,-73.972360,40.756530,-73.987257,40.722395,1,1,1,1,2013,19,11.020122
527373,-74.007343,40.732925,-74.004892,40.709907,1,4,3,5,2013,3,14.030303
...,...,...,...,...,...,...,...,...,...,...,...
189970,-73.974624,40.757523,-73.984226,40.763048,1,6,7,10,2012,18,13.466667
125805,-73.986082,40.722263,-73.991500,40.717302,1,5,16,1,2010,14,8.575189
531478,-74.015780,40.714920,-74.008020,40.717250,6,0,22,12,2014,12,11.852159
329588,-73.991368,40.742375,-73.991370,40.749102,3,2,19,6,2013,23,12.597576


In [67]:
from sklearn.metrics import mean_squared_error
import math

#print(model.score(X_test, Y_test))
mse = mean_squared_error(Y_train, Y_pred.loc[:, ['fare_amount']])
print(math.sqrt(mse))
Y_pred

9.517238425777467


,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dayofweek,day,month,year,hour,fare_amount
361320,-73.990012,40.725420,-74.002815,40.706442,1,5,31,8,2013,14,10.569620
736207,-73.988367,40.740970,-73.994953,40.750312,1,1,24,11,2009,16,11.361594
223908,-73.972995,40.764583,-74.001038,40.729315,5,2,24,6,2009,20,9.342135
905775,-73.972360,40.756530,-73.987257,40.722395,1,1,1,1,2013,19,11.020122
527373,-74.007343,40.732925,-74.004892,40.709907,1,4,3,5,2013,3,14.030303
...,...,...,...,...,...,...,...,...,...,...,...
189970,-73.974624,40.757523,-73.984226,40.763048,1,6,7,10,2012,18,13.466667
125805,-73.986082,40.722263,-73.991500,40.717302,1,5,16,1,2010,14,8.575189
531478,-74.015780,40.714920,-74.008020,40.717250,6,0,22,12,2014,12,11.852159
329588,-73.991368,40.742375,-73.991370,40.749102,3,2,19,6,2013,23,12.597576


In [69]:
from sklearn.tree import DecisionTreeRegressor

XGBR = DecisionTreeRegressor()
XGBR.fit(Y_pred, 
         Y_train)
X_test = X_test.join(data_grouped, on=['year', 'month', 'dayofweek', 'hour'], how="left")
y_pred = XGBR.predict(X_test)

In [70]:
print(XGBR.score(X_test, Y_test))
mse = mean_squared_error(Y_test, XGBR.predict(X_test))
print(math.sqrt(mse))
XGBR.feature_importances_

0.6389471988901847
5.880371566498905


array([0.29693682, 0.11523817, 0.35160524, 0.13349617, 0.00362724,
       0.00835367, 0.01137862, 0.0110352 , 0.02061038, 0.01464195,
       0.03307654])

## Test sur données de test

In [71]:
data_path = "data/test.csv"
data_test = pd.read_csv(data_path)

date = data_test.iloc[:, 1]
date = pd.DatetimeIndex(date)
data_test["dayofweek"] = date.dayofweek
data_test["day"] = date.day
data_test["month"] = date.month
data_test["year"] = date.year
data_test["hour"] = date.hour
X = data_test.iloc[:, 2:]

Y_moyen = X.join(data_grouped, on=['year', 'month', 'dayofweek', 'hour'], how="left")
Y_moyen
#X = data_test.iloc[:, 2:]

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dayofweek,day,month,year,hour,fare_amount
0,-73.973320,40.763805,-73.981430,40.743835,1,1,27,1,2015,13,11.654643
1,-73.986862,40.719383,-73.998886,40.739201,1,1,27,1,2015,13,11.654643
2,-73.982524,40.751260,-73.979654,40.746139,1,5,8,10,2011,11,10.254701
3,-73.981160,40.767807,-73.990448,40.751635,1,5,1,12,2012,21,10.880932
4,-73.966046,40.789775,-73.988565,40.744427,1,5,1,12,2012,21,10.880932
...,...,...,...,...,...,...,...,...,...,...,...
9909,-73.968124,40.796997,-73.955643,40.780388,6,6,10,5,2015,12,13.817870
9910,-73.945511,40.803600,-73.960213,40.776371,6,0,12,1,2015,17,14.427759
9911,-73.991600,40.726608,-73.789742,40.647011,6,6,19,4,2015,20,13.074576
9912,-73.985573,40.735432,-73.939178,40.801731,6,5,31,1,2015,1,11.823529


In [72]:
Y_pred = XGBR.predict(Y_moyen)
Y_pred

array([13. ,  9.5,  3.7, ..., 52. , 20. ,  7.5])

In [73]:
submission = pd.DataFrame(
    {'key': data_test.key, 'fare_amount': Y_pred},
    columns = ['key', 'fare_amount'])
submission.to_csv('submission.csv', index = False)
submission

,key,fare_amount
0,2015-01-27 13:08:24.0000002,13.0
1,2015-01-27 13:08:24.0000003,9.5
2,2011-10-08 11:53:44.0000002,3.7
3,2012-12-01 21:12:12.0000002,6.9
4,2012-12-01 21:12:12.0000003,12.5
...,...,...
9909,2015-05-10 12:37:51.0000002,6.0
9910,2015-01-12 17:05:51.0000001,14.0
9911,2015-04-19 20:44:15.0000001,52.0
9912,2015-01-31 01:05:19.0000005,20.0


In [74]:
import os
os.system('kaggle competitions submit -c new-york-city-taxi-fare-prediction -f submission.csv -m "Message"')

0